# Section 4 - Container images, where to find them and how to build them
## 35 - What's an image and what is it not
An image is a set of application binaries, their dependancies as well as the metadata on the image and how it should run.  
It doesn't contain an OS, the host on which the application runs will provide the kernel/OS/Drives


## 36- The mighty hub: using docker hub registry images 

The image registry can be found at hub.docker.com

Each application can have multiple images on the registry (e.g: nginx has ~90,000 images). To begin with, an easy choice  
is to start with the official images that are curated by the team at Docker (who usualy work hand in hand with the team that created the software).

You can find documentation on how to run the image within its page on the hub.

There are multiple versions of most official images. Each version can have more than one tag.

## 37- Images and their layers: discover the image cache

Images don't just come as a single piece of data. They're usualy stacks of code that build on top of each other  
(much like commits). This can be seen when downloading new images and there 

They are designed using the union file concept of making layers of changes.

In [ ]:
%%bash
docker image history nginx:latest

The above is a history of the image layers. Every image start with a blank base layer called "scratch", every change since then on the image file system is an added layer. Every new layer has its own unique hash code (sha265).

Images being stacks of layers can "share" cached layers that are common across images (e.g: Two images can use the same scratch layer of Debian Jessie)

In deploying containers, if you modify one of the files in the base layer that is shared among containers. Only the difference is copied out. This is known as copy on write, the differencing elements in the layers are copied into the container that does the changes (not affecting the other containers).

The docker inspect command return metadata on the image in JSON format.

In [ ]:
%%bash
docker inspect nginx

## 38- Image taggin and pushing to Docker hub

Image tag syntax: \<user>/\<repo>:\<tag>

A tag is a pointer to a specific image commit (IMAGE ID). It a many to one relation between TAG and IMAGE ID (N:1)

In [ ]:
%%bash

docker image tag --help

You can retag new tags to existing images

In [ ]:
%%bash
docker image tag nginx karimitn/nginx

In [ ]:
%%bash
#you can see the new tag for karimitn/nginx having the same IMAGE ID as the original nginx image
docker image ls 

In [ ]:
%%bash
#The new tag won't show in Docker Hub untill you run a push command
docker image push karimitn/nginx

## 39- Building Images: The Dockerfile Basics
Note: Use the dockerfile-sample-1 folder for this section

The default name for such files is Dockerfile but any other name can be supplied. You can use the flag -f to specify which file you want to use.

Example:\
docker build -f some-dockerfile

Each stanza in the Dockerfile is an actual layer and thus the order in which they are run matters a lot.

### Structure: 

- FROM command is required, it tells docker which image layer this is building up from
- ENV is used to set environemnt variables
- RUN is used to run shell command during the build process (usualy installing packages, zipping, file edits,etc)
- EXPOSE set which ports are exposed (by default no ports are exposed in a container to a virtual network)
- CMD is the final command that is added (it is used every time a new container is launched from the image or restart a stopped container)




In [ ]:
%%bash
pwd
cd ./udemy-docker-mastery/dockerfile-sample-1/
pwd
ls -ltra
cat Dockerfile


## 40- Building Images: Running Docker Buils

When you run the above Dockerfile it will pull the Debian:Jessie image from Docker Hub into the cache then it will execute the subsequent commands inside the Docker Engine and cache each of those layers

In [ ]:
%%bash
cd ./udemy-docker-mastery/dockerfile-sample-1/
docker image build -t customnginx .